---
title: Create XArray Dataset
description: "create the raw dataset xarray (nc) file through `database_etl`"
project: parafac2
conclusion: "while execution of database_etl to produce the expected dataset was successful, we found that xarray datasets wernt queryable across variables as was expected. Efforts to recreate the img data within duckdb were fruitless, as expected, however we did find that judicious use of double quotes and string formatting enabled us to construct proper tidy tables with wavelength integer column labels, but unfortunately unidentified (probably memory ) problems prevented ingestion of data in this method to be practical as it would have taken 32 minutes if inserted row by row. Recommendation is to construct an intermediary between sql queries and the xarray dataset or the stored parquet files. Probably the latter as it removes one complication."
status: closed
cdt: 2024-09-25T16:17:38
---

In [ ]:
%reload_ext autoreload
%autoreload 2

from database_etl import etl_pipeline_raw, get_data
import polars as pl

from pca_analysis.definitions import (
    RAW_LIB_DIR,
    DIRTY_ST,
    CT_UN,
    CT_PW,
    DB_PATH_UV,
    NC_RAW,
)

import duckdb as db
import xarray as xr

con = db.connect(DB_PATH_UV)
con.close()
overwrite = True

if overwrite:
    etl_pipeline_raw(
        data_dir=RAW_LIB_DIR,
        dirty_st_path=DIRTY_ST,
        ct_pw=CT_PW,
        ct_un=CT_UN,
        con=con,
        overwrite=True,
        run_extraction=True,
        excluded_samples=[
            {
                "samplecode": "2021-debortoli-cabernet-merlot_avantor",
                "reason": "aborted run",
            }
        ],
    )

    dset.to_netcdf(NC_RAW)
else:
    dset = xr.open_dataset(NC_RAW)

dset = dset.assign_coords({"wavelength": dset["wavelength"].astype(int)})
dset.pipe(display)


And as a demonstration.. the red shiraz.

In [ ]:
# shiraz = dset.sel(color="red", varietal="shiraz", wavelength=256)
# #


In [ ]:
# shiraz = dset.sel(id=["e56c4dcd-2847-4d34-b457-743be10b0608"])
# shiraz
# con.close()


It actually appears that you cant actually subset xarray Datasets..

Considering that the data is already setup in the database, I think it would be better to go back to SQL first..

In [ ]:
# df = (
#     dset.img.sel(id="e56c4dcd-2847-4d34-b457-743be10b0608").to_dataframe().reset_index()
# )
# id = df["id"][0]
# df_ = df.drop("id", axis=1)
# tidy_df = df_.set_index("mins").pivot(columns="wavelength", values="img")

# db.sql(
#     """--sql
#     select "190" from tidy_df
#     """
# )


In [ ]:
# from database_etl.etl.sql.to_xr.sql_to_xr import get_imgs_as_dict

# result = get_imgs_as_dict(con=con, m=7800)
# list(result.values())[0]


In [ ]:
# wavelengths = list(result.values())[0].columns


In [ ]:
# tidy_imgs = (
#     pl.from_pandas(img.assign(**{"id": id}).reset_index()) for id, img in result.items()
# )


Can insert, but then cant have primary keys.

In [ ]:
# del dset
# del df
# del df_
# del result


In [ ]:
# def create_t2(con, wavelengths):
#     wavelength_col_decs = ",".join(
#         [f'"{x}" float' for x in wavelengths if x not in ["mins", "id"]]
#     )

#     con.sql(
#         f"""--sql
#     drop table t2;
#     create or replace table t2 (
#         id varchar references chm(id),
#         mins float,
#         {wavelength_col_decs},
#         primary key (id, mins)
#         );
#     """
#     )


# def insert_img(con, img, wavelengths):
#     for x in img.partition_by("mins"):
#         # display(x)
#         con.sql(
#             f"""--sql
#         insert into t2
#             select
#                 id,
#                 mins,
#                 {",".join([f'"{x}"' for x in wavelengths])}
#             from
#                 x
#         """
#         )


# def insert_imgs(con, imgs, wavelengths):
#     for idx, img in enumerate(imgs):
#         print(idx, img["id"][0])
#         insert_img(con=con, img=img, wavelengths=wavelengths)


# def create_tidy_img_tbl(con, imgs, wavelengths) -> None:
#     create_tidy_img_tbl(con=con, imgs=imgs, wavelengths=wavelengths)
#     insert_imgs(con, imgs, wavelengths=wavelengths)


In [ ]:
# wavelengths = [x for x in wavelengths if x not in ["id", "mins"]]
# shortened_wavelengths = wavelengths[:5]
# print(shortened_wavelengths)
# create_t2(con=con, wavelengths=shortened_wavelengths)
# insert_imgs(con, tidy_imgs, wavelengths=shortened_wavelengths)

# con.sql(
#     """--sql
# select
#     *
# from
#     t2
# limit 10
# """
# ).pl()


Ok, as promising as this is, its obviosuly making something upset. It's just not worth the effort to make this work.


# Conclusion

While it did look possible to create a tidy image table, insertion of data proved impossible, resulting in the kernel dying more often than not. Evidently the mechanism wasnt intended for this volume of insertion.

We could try reading the parquet files directly, or inserting one row at a time..

Ok look we could play around with it like this all week, but its evident that duckdb doesnt like my data. End of the day, we're not looking to query the raw data anyway, and furthermore 99% of the wavelengths dont contain any useful information anyway.

Create a rough module to translate the results of a query to filepaths then return the selected data as a list/generator. Load it into a tensor and fire away. Anything else is a WASTE OF TIME. Or query on the dset, selecting by ID.
